## *NBC Overseas Correspondent Classifier*

In [ ]:
import os
import shutil
import random
import numpy as np
from pyAudioAnalysis import audioTrainTest as aT
from pyAudioAnalysis import audioSegmentation as aS

In [ ]:
# Creating new directories to keep our files organized

!mkdir /home/sharedfolder/NBC_DDay_News
!mkdir /home/sharedfolder/NBC_DDay_News_Clips
!mkdir /home/sharedfolder/NBC_DDay_News_Clips_Subset

In [ ]:
# Let's change our working directory to "NBC_DDay_Complete_Broadcast" and create a list of filenames.

os.chdir('/home/sharedfolder/NBC_DDay_Complete_Broadcast')
filenames=os.listdir('./')
filenames[:10]               # displays first 10 filenames

### *Task:*

Move all files whose names contain "News" into the directory **`/home/sharedfolder/NBC_DDay_News`**.

**Hint:** You can either do this in the terminal or in Python. **`shutil.move("filename.mp3", "/path/to/directory/")`** will move a file to another directory, just like **`mv`** does in Bash.

If you choose to use Python, one option is to use the list comprehension feature. You may want to create and view your list of "News" filenames before you actually move them.

### *Splitting recordings into 30-second segments*

Next we'll use **`ffmpeg`** to split each file into 30-second WAV chunks (which are quicker and more convenient to work with than full recordings). 

In the terminal, **`cd`** to the directory **`NBC_DDay_News_Clips`** and run the following command, which will create numbered 30-second segments whose filenames end with **`_clip.wav`**. The process may take a few minutes to complete.

```
for f in *.mp3; do
ffmpeg -i $f -f segment -segment_time 30 $f.%04d_clip.wav ;
done
```

### *Task:*

Use Python to randomly choose around 300 of the files you just created and move them to the directory **`NBC_DDay_News_Clips_Subset`**.

First you'll need to change your current directory to **`/home/sharedfolder/NBC_DDay_News_Clips`** with **`os.chdir(...)`**, then use **`os.listdir('./')`** to create a list of filenames like we did in the third cell of this notebook.

**Hint:** Only include filenames that end with **`_clip.wav`**. We only want to work with our 30-second clips in the next step, not the original recordings.

Try running the cell below a few times to see how **`random.sample(...)`** chooses items randomly from a list. 

Once you've created a list of randomly chosen filenames, you can use **`shutil.move(..., ...)`**  to move them to the directory **`NBC_DDay_News_Clips_Subset`**, just like we did a few cells back.


In [ ]:
random.sample([1,2,3,4,5,6,7,8,9,10], 4)

The program **`sox`**, known as "the Swiss Army knife of sound processing programs," can create high-quality spectrograms and save them as PNG image files.

Back in the terminal, **`cd`** to the directory **`NBC_DDay_News_Clips_Subset`** and run the following command to create spectrograms for every file in the set.

```
for f in *_clip.wav; do
sox $f -n spectrogram -x 1600 -y 513 -r -o $basename.png;
done
```

## *Sorting things out*

We should now have a few hundred 30-second WAVs and corresponding PNG spectrograms in the same directory, **`NBC_DDay_News_Clips_Subset`**. Next we'll sort through the PNG files to find the really noisy ones from foreign correpondents — but first we need to create the directories where we'll put them.

In [ ]:
# Directories where we'll put our two training classes

!mkdir /home/sharedfolder/noisy
!mkdir /home/sharedfolder/not_noisy

Open **`sharedfolder`** on your desktop and start opening spectrogram files with whatever program is most convenient. If a file looks really noisy, drag the corresponding WAV file to the **`noisy`** directory. If not, put the WAV file in **`not_noisy`**.

If you can't decide where to put a file, take a quick listen and make a decision. This is your classifier; you call the shots. If you come across a file that contains music, just ignore it and move on.

Once you have 20 or 30 WAVs in each class, you're ready to train your model.

## *Training your classifier*

Set your working directory to **`/home/sharedfolder`** and create a directory for your new model files.

In [ ]:
os.chdir('/home/sharedfolder')

!mkdir models

The following command will train an SVM classifier using the Python package **`pyAudioAnalysis`**, which it will store in the directory **`models`**. Fingers crossed!

The list argument **`['not_noisy','noisy']`** in the command below actually points to the directories **`noisy`** and **`not_noisy`**. Because of the order we've chosen, the model will output 0 (zero) for "not_noisy" and 1 for "noisy."

In [ ]:
aT.featureAndTrain(['not_noisy','noisy'], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "svm", "models/svm_NBC_noise", False)

Now let's test the classifier on a single file and see what we get. This one happens to be a foreign correspondent with lots of noise in the background.

In [ ]:
wav_path = '/home/sharedfolder/NBC_DDay_News_Clips/CBD-440606_NBC0330-News.mp3.0014_clip.wav'

classifier_model_path = '/home/sharedfolder/models/svm_NBC_noise'

output, classesAll, acc, CM = aS.mtFileClassification(wav_path, classifier_model_path, "svm")

output

If the output is a list of all ones, your clssifier worked! Give yourself a pat on the back.

Because we ran the classifier on a 30-second file, we get back 30 classifications, one for each second of audio.

Find a file you know isn't noisy and see if the classifier gets it right. (Just make sure it isn't one that you included in your training set.)

## *Task:*

Choose 100 files at random from the directory **`NBC_DDay_News_Clips`** and classify them all, using **`print(...)`** to output each filename alongside its classificaiton.

## *Task:*


Create a function called `**is_noisy(...)**` that take a WAV pathname as an argument and returns a decision for the whole file.

**Hint:** The **`np.mean(...)`** function, which is part of the **`numpy`** package, will help you get from a list of numbers to a binary decision: 0 or 1, "noisy" or "not_noisy." Run the cell below for a demonstration.

In [ ]:
np.mean([0,0,0,0,0,0,1,1,0,1,0,0,1,1,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0])

## *Bonus task:*

You're probably getting lots of music mixed in with your "noisy" classifications.

Download the source files for **[pyAudioAnalysis](https://github.com/tyiannak/pyAudioAnalysis)** and add the following trained model files (located in that package's **``data``** directory) to your **`models`** directory.

```
svmSM
svmSM.arff
svmSMMEANS
```

Now write a function called **`is_music(...)`** that takes a WAV pathname as an argument and returns a binary classification.

Now create a new function called **`is_foreign_correspondent(...)`** that checks whether a file is "noisy" as well as whether it's music. Test it out on a new batch of random clips.

## *Bonus task:*

When you randomly choose clips to classify, check their filenames to make sure they don't come from the same files you used in your 

## *Bonus task:*

Add an audio player widget to your notebook that lets you listen to each clip alongside its classification. Go ahead and crib some code from the [Classify_and_Play](https://github.com/hipstas/audio-tagging-toolkit/blob/master/scripts/Classify_and_Play.zip) notebook we looked at yesterday.

## *Bonus task:*

Write your own k-fold cross-validation system to quantify how well your model performs.